In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Wczytanie modelu YOLO
model = YOLO('yolov8n.pt')  # Model lekki, można użyć np. 'yolov8m.pt' dla lepszej dokładności

# Ścieżki do plików wejściowego i wyjściowego
#input_path = 'F://BurzaVSZryw_mini.mp4'
input_path = '/media/512GB_ext/BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 2704
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 900
fps = cap.get(cv2.CAP_PROP_FPS)

# Obliczenie rozmiaru kadru dla proporcji 16:9
crop_height = height  # 900
crop_width = int(crop_height * 16 / 9)  # 1600

# Inicjalizacja zapisu wideo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (crop_width, crop_height))

# Parametry wygładzania pozycji kadru
window_size = 30  # Liczba klatek do średniej kroczącej
positions = []

def calculate_center(detections):
    """Oblicza centrum akcji na podstawie wykrytych obiektów."""
    if not detections:
        return width // 2  # Domyślnie środek, jeśli brak detekcji
    centroids = []
    for det in detections:
        x1, y1, x2, y2 = det[:4]
        centroids.append((x1 + x2) / 2)  # Pozycja X środka bounding boxa
    if centroids:
        return sum(centroids) / len(centroids)
    return width // 2

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Detekcja obiektów w klatce
    results = model(frame)
    detections = results[0].boxes.data.cpu().numpy()  # Format: [x1, y1, x2, y2, confidence, class]
    
    # Filtrowanie detekcji: zawodnicy (class 0 - 'person') i piłka (class 32 - 'sports ball')
    players = [det for det in detections if int(det[5]) == 0]
    balls = [det for det in detections if int(det[5]) == 32]
    
    # Obliczenie centrum akcji
    if balls:
        # Jeśli wykryto piłkę, użyj jej pozycji
        center_x = calculate_center(balls)
    else:
        # W przeciwnym razie użyj centroidu zawodników
        center_x = calculate_center(players)
    
    # Wygładzenie pozycji kadru
    positions.append(center_x)
    if len(positions) > window_size:
        positions.pop(0)
    smoothed_center_x = sum(positions) / len(positions)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    right = left + crop_width
    
    # Przycięcie klatki
    cropped_frame = frame[:, left:right]
    
    # Zapis przyciętej klatki
    out.write(cropped_frame)

# Zwolnienie zasobów
cap.release()
out.release()
#cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from moviepy.editor import VideoFileClip, VideoClip
import os
from datetime import datetime
import logging

# Wyłączenie szczegółowych logów YOLOv8
logging.getLogger("ultralytics").setLevel(logging.ERROR)
# Sprawdzenie dostępności GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Wczytanie modelu YOLO
model = YOLO('yolov8n.pt').to(device)  # Przeniesienie modelu na GPU, jeśli dostępne

# Ścieżki do plików
input_path = '/media/512GB_ext/BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'
temp_video_path = f'temp_output_{datetime.now().strftime("%Y%m%d_%H%M%S")}.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Docelowy rozmiar kadru (16:9, wysokość 720)
crop_height = 720
crop_width = int(crop_height * 16 / 9)  # 1280

# Parametry wygładzania
window_size = 15
positions_x = []
positions_y = []
smoothing_factor = 0.1

# Inicjalizacja zapisu tymczasowego wideo bez dźwięku
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(temp_video_path, fourcc, fps, (crop_width, crop_height))

def split_frame(frame, tile_size=640):
    """Dzieli obraz na kafelki dla lepszej detekcji małych obiektów."""
    tiles = []
    h, w = frame.shape[:2]
    for y in range(0, h, tile_size):
        for x in range(0, w, tile_size):
            tile = frame[y:min(y + tile_size, h), x:min(x + tile_size, w)]
            tiles.append((tile, x, y))
    return tiles

def calculate_center(detections, frame_width, frame_height):
    """Oblicza centrum akcji, priorytetyzując piłkę."""
    if not detections:
        return frame_width // 2, frame_height // 2
    balls = [det for det in detections if int(det[5]) == 32]  # 'sports ball'
    players = [det for det in detections if int(det[5]) == 0]  # 'person'
    
    if balls:
        x1, y1, x2, y2 = balls[0][:4]
        return (x1 + x2) / 2, (y1 + y2) / 2
    elif players:
        centroids_x = [(det[0] + det[2]) / 2 for det in players]
        centroids_y = [(det[1] + det[3]) / 2 for det in players]
        return sum(centroids_x) / len(centroids_x), sum(centroids_y) / len(centroids_y)
    return frame_width // 2, frame_height // 2

prev_center_x, prev_center_y = width // 2, height // 2
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Podział na kafelki dla detekcji
    tiles = split_frame(frame, tile_size=640)
    detections = []
    for tile, tile_x, tile_y in tiles:
        results = model(tile, classes=[0, 32], device=device)
        tile_detections = results[0].boxes.data.cpu().numpy()
        for det in tile_detections:
            det[0] += tile_x
            det[2] += tile_x
            det[1] += tile_y
            det[3] += tile_y
            detections.append(det)
    
    # Obliczenie centrum akcji
    center_x, center_y = calculate_center(detections, width, height)
    
    # Wygładzanie liniowe
    center_x = prev_center_x * (1 - smoothing_factor) + center_x * smoothing_factor
    center_y = prev_center_y * (1 - smoothing_factor) + center_y * smoothing_factor
    prev_center_x, prev_center_y = center_x, center_y
    
    # Wygładzanie średnią kroczącą
    positions_x.append(center_x)
    positions_y.append(center_y)
    if len(positions_x) > window_size:
        positions_x.pop(0)
        positions_y.pop(0)
    smoothed_center_x = sum(positions_x) / len(positions_x)
    smoothed_center_y = sum(positions_y) / len(positions_y)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    top = int(max(0, min(smoothed_center_y - crop_height / 2, height - crop_height)))
    right = left + crop_width
    bottom = top + crop_height
    
    # Przycięcie klatki
    cropped_frame = frame[top:bottom, left:right]
    
    # Skalowanie do 1280x720, jeśli kadr nie jest dokładny
    if cropped_frame.shape != (crop_height, crop_width, 3):
        cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height), interpolation=cv2.INTER_AREA)
    
    # Zapis klatki do tymczasowego pliku
    out.write(cropped_frame)

# Zwolnienie zasobów OpenCV
cap.release()
out.release()
cv2.destroyAllWindows()

# Dodanie dźwięku za pomocą moviepy
print("Adding audio to the output video...")
input_clip = VideoFileClip(input_path)
temp_clip = VideoFileClip(temp_video_path)

# Ustawienie dźwięku z oryginalnego wideo
final_clip = temp_clip.set_audio(input_clip.audio)

# Zapis finalnego wideo z dźwiękiem
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Zamknięcie klipów
input_clip.close()
temp_clip.close()
final_clip.close()

# Usunięcie tymczasowego pliku
os.remove(temp_video_path)
print("Output video with audio saved successfully.")

Using device: cuda


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from moviepy.editor import VideoFileClip
import os
from datetime import datetime
from tqdm import tqdm
import logging

# Wyłączenie szczegółowych logów YOLOv8
logging.getLogger("ultralytics").setLevel(logging.ERROR)
# Sprawdzenie dostępności GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Wczytanie modelu YOLO
#model = YOLO('yolov8n.pt').to(device)  # Przeniesienie modelu na GPU, jeśli dostępne
model = YOLO('yolov9c.pt').to(device)
model.set_classes = (["player","referee", "ball"])
# Ścieżki do plików
input_path = '/media/512GB_ext/BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'
temp_video_path = f'temp_output_{datetime.now().strftime("%Y%m%d_%H%M%S")}.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Docelowy rozmiar kadru (16:9, wysokość 720)
crop_height = 720
crop_width = int(crop_height * 16 / 9)  # 1280

# Parametry wygładzania
window_size = 15
positions_x = []
positions_y = []
smoothing_factor = 0.1

# Parametry dla skupienia na piłce
frames_without_ball_threshold = int(fps * 3)  # 3 sekundy
frames_without_ball = 0
last_ball_position = None

# Inicjalizacja zapisu tymczasowego wideo bez dźwięku
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(temp_video_path, fourcc, fps, (crop_width, crop_height))

def resize_frame(frame, scale=0.5):
    """Zmniejsza obraz o podany współczynnik skali."""
    return cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

def scale_detections(detections, scale):
    """Przeskalowuje współrzędne detekcji do oryginalnych wymiarów."""
    for det in detections:
        det[0] /= scale  # x1
        det[2] /= scale  # x2
        det[1] /= scale  # y1
        det[3] /= scale  # y2
    return detections

def draw_detections(frame, detections, left, top, right, bottom):
    """Rysuje bounding boxy i nazwy klas na przyciętym obrazie."""
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        # Sprawdzenie, czy detekcja jest w przyciętym obszarze
        if x1 < right and x2 > left and y1 < bottom and y2 > top:
            # Przesunięcie współrzędnych do przyciętego obszaru
            draw_x1 = int(max(x1 - left, 0))
            draw_y1 = int(max(y1 - top, 0))
            draw_x2 = int(min(x2 - left, crop_width))
            draw_y2 = int(min(y2 - top, crop_height))
            label = model.names[int(cls)]
            cv2.rectangle(frame, (draw_x1, draw_y1), (draw_x2, draw_y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (draw_x1, draw_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def calculate_center(detections, frame_width, frame_height, use_ball=True):
    """Oblicza centrum akcji, priorytetyzując piłkę lub zawodników."""
    if use_ball:
        balls = [det for det in detections if int(det[5]) == 32]  # 'sports ball'
        if balls:
            x1, y1, x2, y2 = balls[0][:4]
            return (x1 + x2) / 2, (y1 + y2) / 2
    players = [det for det in detections if int(det[5]) == 0]  # 'person'
    if players:
        centroids_x = [(det[0] + det[2]) / 2 for det in players]
        centroids_y = [(det[1] + det[3]) / 2 for det in players]
        return sum(centroids_x) / len(centroids_x), sum(centroids_y) / len(centroids_y)
    return frame_width // 2, frame_height // 2

prev_center_x, prev_center_y = width // 2, height // 2

# Pasek postępu
pbar = tqdm(total=total_frames, desc="Processing frames")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Zmniejszenie obrazu o połowę dla detekcji
    small_frame = resize_frame(frame, scale=0.5)
    small_height, small_width = small_frame.shape[:2]
    
    # Detekcja na pomniejszonym obrazie
    results = model(small_frame, classes=[0, 32], device=device)
    detections = results[0].boxes.data.cpu().numpy()
    
    # Przeskalowanie detekcji do oryginalnych wymiarów
    detections = scale_detections(detections, scale=0.5)
    
    # Sprawdzenie, czy wykryto piłkę
    balls = [det for det in detections if int(det[5]) == 32]
    if balls:
        frames_without_ball = 0
        last_ball_position = balls[0][:4]
    else:
        frames_without_ball += 1
    
    # Decyzja, czy skupić się na piłce czy zawodnikach
    if frames_without_ball <= frames_without_ball_threshold and last_ball_position is not None:
        # Użyj ostatniej znanej pozycji piłki
        x1, y1, x2, y2 = last_ball_position
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
    else:
        # Użyj pozycji zawodników
        center_x, center_y = calculate_center(detections, width, height, use_ball=False)
    
    # Wygładzanie liniowe
    center_x = prev_center_x * (1 - smoothing_factor) + center_x * smoothing_factor
    center_y = prev_center_y * (1 - smoothing_factor) + center_y * smoothing_factor
    prev_center_x, prev_center_y = center_x, center_y
    
    # Wygładzanie średnią kroczącą
    positions_x.append(center_x)
    positions_y.append(center_y)
    if len(positions_x) > window_size:
        positions_x.pop(0)
        positions_y.pop(0)
    smoothed_center_x = sum(positions_x) / len(positions_x)
    smoothed_center_y = sum(positions_y) / len(positions_y)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    top = int(max(0, min(smoothed_center_y - crop_height / 2, height - crop_height)))
    right = left + crop_width
    bottom = top + crop_height
    
    # Przycięcie oryginalnego obrazu
    cropped_frame = frame[top:bottom, left:right]
    
    # Narysowanie detekcji na przyciętym obrazie
    draw_detections(cropped_frame, detections, left, top, right, bottom)
    
    # Skalowanie do 1280x720, jeśli kadr nie jest dokładny
    if cropped_frame.shape != (crop_height, crop_width, 3):
        cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height), interpolation=cv2.INTER_AREA)
    
    # Zapis klatki do tymczasowego pliku
    out.write(cropped_frame)
    
    # Aktualizacja paska postępu
    pbar.update(1)

# Zamknięcie paska postępu
pbar.close()

# Zwolnienie zasobów OpenCV
cap.release()
out.release()
cv2.destroyAllWindows()

# Dodanie dźwięku za pomocą moviepy
print("Adding audio to the output video...")
input_clip = VideoFileClip(input_path)
temp_clip = VideoFileClip(temp_video_path)

# Ustawienie dźwięku z oryginalnego wideo
final_clip = temp_clip.set_audio(input_clip.audio)

# Zapis finalnego wideo z dźwiękiem
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Zamknięcie klipów
input_clip.close()
temp_clip.close()
final_clip.close()

# Usunięcie tymczasowego pliku
os.remove(temp_video_path)
print("Output video with audio saved successfully.")

Using device: cuda


Processing frames: 100%|██████████| 959/959 [00:25<00:00, 37.39it/s]


Adding audio to the output video...
Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
Output video with audio saved successfully.


In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from moviepy.editor import VideoFileClip
import os
from datetime import datetime
from tqdm import tqdm
import logging

# Wyłączenie szczegółowych logów YOLO
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# Sprawdzenie dostępności GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Wczytanie modelu YOLO
model = YOLO('yolov9c.pt').to(device)  # Przeniesienie modelu na GPU, jeśli dostępne

# Mapowanie indeksów klas na nazwy (zamiast zmiany model.names)
class_names = {0: "player", 1: "referee", 2: "ball"}

# Ścieżki do plików
input_path = '/media/512GB_ext/BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'
temp_video_path = f'temp_output_{datetime.now().strftime("%Y%m%d_%H%M%S")}.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Docelowy rozmiar kadru (16:9, wysokość 720)
crop_height = 720
crop_width = int(crop_height * 16 / 9)  # 1280
scale = 1  # Skala do zmniejszenia obrazu

# Parametry wygładzania
window_size = 15
positions_x = []
positions_y = []
smoothing_factor = 0.1

# Parametry dla skupienia na piłce
frames_without_ball_threshold = int(fps * 3)  # 3 sekundy
frames_without_ball = 0
last_ball_position = None

# Inicjalizacja zapisu tymczasowego wideo bez dźwięku
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(temp_video_path, fourcc, fps, (crop_width, crop_height))

def resize_frame(frame, scale=scale):
    """Zmniejsza obraz o podany współczynnik skali."""
    return cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

def scale_detections(detections, scale):
    """Przeskalowuje współrzędne detekcji do oryginalnych wymiarów."""
    for det in detections:
        det[0] /= scale  # x1
        det[2] /= scale  # x2
        det[1] /= scale  # y1
        det[3] /= scale  # y2
    return detections

def draw_detections(frame, detections, left, top, right, bottom):
    """Rysuje bounding boxy i nazwy klas na przyciętym obrazie."""
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        # Sprawdzenie, czy detekcja jest w przyciętym obszarze
        if x1 < right and x2 > left and y1 < bottom and y2 > top:
            # Przesunięcie współrzędnych do przyciętego obszaru
            draw_x1 = int(max(x1 - left, 0))
            draw_y1 = int(max(y1 - top, 0))
            draw_x2 = int(min(x2 - left, crop_width))
            draw_y2 = int(min(y2 - top, crop_height))
            label = class_names[int(cls)]  # Użycie mapowania klas
            cv2.rectangle(frame, (draw_x1, draw_y1), (draw_x2, draw_y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (draw_x1, draw_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def calculate_center(detections, frame_width, frame_height, use_ball=False):
    """Oblicza centrum akcji, priorytetyzując piłkę lub zawodników/sędziów."""
    if use_ball:
        balls = [det for det in detections if int(det[5]) == 2]  # 'ball'
        if balls:
            x1, y1, x2, y2 = balls[0][:4]
            return (x1 + x2) / 2, (y1 + y2) / 2
    players_referees = [det for det in detections if int(det[5]) in [0, 1]]  # 'player' lub 'referee'
    if players_referees:
        centroids_x = [(det[0] + det[2]) / 2 for det in players_referees]
        centroids_y = [(det[1] + det[3]) / 2 for det in players_referees]
        return sum(centroids_x) / len(centroids_x), sum(centroids_y) / len(centroids_y)
    return frame_width // 2, frame_height // 2

prev_center_x, prev_center_y = width // 2, height // 2

# Pasek postępu
pbar = tqdm(total=total_frames, desc="Processing frames")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Zmniejszenie obrazu o połowę dla detekcji
    small_frame = resize_frame(frame, scale=scale)
    small_height, small_width = small_frame.shape[:2]
    
    # Detekcja na pomniejszonym obrazie
    results = model(small_frame, classes=[0, 1, 2], device=device)  # Ogranicz do 'player', 'referee', 'ball'
    detections = results[0].boxes.data.cpu().numpy()
    
    # Przeskalowanie detekcji do oryginalnych wymiarów
    detections = scale_detections(detections, scale=scale)
    
    # Sprawdzenie, czy wykryto piłkę
    balls = [det for det in detections if int(det[5]) == 2]
    if balls:
        frames_without_ball = 0
        last_ball_position = balls[0][:4]
    else:
        frames_without_ball += 1
    
    # Decyzja, czy skupić się na piłce czy zawodnikach/sędziach
    #if frames_without_ball <= frames_without_ball_threshold and last_ball_position is not None:
        # Użyj ostatniej znanej pozycji piłki
    #    x1, y1, x2, y2 = last_ball_position
    #    center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
    #else:
        # Użyj pozycji zawodników/sędziów
    center_x, center_y = calculate_center(detections, width, height, use_ball=False)
    
    # Wygładzanie liniowe
    center_x = prev_center_x * (1 - smoothing_factor) + center_x * smoothing_factor
    center_y = prev_center_y * (1 - smoothing_factor) + center_y * smoothing_factor
    prev_center_x, prev_center_y = center_x, center_y
    
    # Wygładzanie średnią kroczącą
    positions_x.append(center_x)
    positions_y.append(center_y)
    if len(positions_x) > window_size:
        positions_x.pop(0)
        positions_y.pop(0)
    smoothed_center_x = sum(positions_x) / len(positions_x)
    smoothed_center_y = sum(positions_y) / len(positions_y)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    top = int(max(0, min(smoothed_center_y - crop_height / 2, height - crop_height)))
    right = left + crop_width
    bottom = top + crop_height
    
    # Przycięcie oryginalnego obrazu
    cropped_frame = frame[top:bottom, left:right]
    
    # Narysowanie detekcji na przyciętym obrazie
    draw_detections(cropped_frame, detections, left, top, right, bottom)
    
    # Skalowanie do 1280x720, jeśli kadr nie jest dokładny
    if cropped_frame.shape != (crop_height, crop_width, 3):
        cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height), interpolation=cv2.INTER_AREA)
    
    # Zapis klatki do tymczasowego pliku
    out.write(cropped_frame)
    
    # Aktualizacja paska postępu
    pbar.update(1)

# Zamknięcie paska postępu
pbar.close()

# Zwolnienie zasobów OpenCV
cap.release()
out.release()
cv2.destroyAllWindows()

# Dodanie dźwięku za pomocą moviepy
print("Adding audio to the output video...")
input_clip = VideoFileClip(input_path)
temp_clip = VideoFileClip(temp_video_path)

# Ustawienie dźwięku z oryginalnego wideo
final_clip = temp_clip.set_audio(input_clip.audio)

# Zapis finalnego wideo z dźwiękiem
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Zamknięcie klipów
input_clip.close()
temp_clip.close()
final_clip.close()

# Usunięcie tymczasowego pliku
os.remove(temp_video_path)
print("Output video with audio saved successfully.")

Using device: cuda


Processing frames: 100%|██████████| 959/959 [00:25<00:00, 38.29it/s]


Adding audio to the output video...
Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
Output video with audio saved successfully.


In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from moviepy.editor import VideoFileClip
import os
from datetime import datetime
from tqdm import tqdm
import logging

# Wyłączenie szczegółowych logów YOLO
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# Sprawdzenie dostępności GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Wczytanie modelu YOLO
model = YOLO('yolov9c.pt').to(device)  # Przeniesienie modelu na GPU, jeśli dostępne

# Mapowanie indeksów klas na nazwy
class_names = {0: "player", 1: "referee", 2: "ball"}

# Ścieżki do plików
input_path = 'input_01.mp4'
output_path = 'test.mp4'
temp_video_path = f'temp_output_{datetime.now().strftime("%Y%m%d_%H%M%S")}.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Docelowy rozmiar kadru (16:9, wysokość 720)
crop_height = 720
crop_width = int(crop_height * 16 / 9)  # 1280

# Parametry wygładzania
window_size = 15
positions_x = []
positions_y = []
smoothing_factor = 0.1

# Parametry dla skupienia na piłce
frames_without_ball_threshold = int(fps * 3)  # 3 sekundy
frames_without_ball = 0
last_ball_position = None
prev_ball_centroid = None
movement_threshold = 10  # Piksele - próg ruchu piłki

# Inicjalizacja zapisu tymczasowego wideo bez dźwięku
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(temp_video_path, fourcc, fps, (crop_width, crop_height))

def resize_frame(frame, scale=0.5):
    """Zmniejsza obraz o podany współczynnik skali."""
    return cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

def scale_detections(detections, scale):
    """Przeskalowuje współrzędne detekcji do oryginalnych wymiarów."""
    for det in detections:
        det[0] /= scale  # x1
        det[2] /= scale  # x2
        det[1] /= scale  # y1
        det[3] /= scale  # y2
    return detections

def draw_detections(frame, detections, left, top, right, bottom):
    """Rysuje bounding boxy i nazwy klas na przyciętym obrazie dla wszystkich klas."""
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if x1 < right and x2 > left and y1 < bottom and y2 > top:
            draw_x1 = int(max(x1 - left, 0))
            draw_y1 = int(max(y1 - top, 0))
            draw_x2 = int(min(x2 - left, crop_width))
            draw_y2 = int(min(y2 - top, crop_height))
            label = class_names[int(cls)]  # Użycie mapowania klas
            color = (0, 255, 0) if int(cls) == 2 else (255, 0, 0)  # Zielony dla piłki, niebieski dla graczy/sędziów
            cv2.rectangle(frame, (draw_x1, draw_y1), (draw_x2, draw_y2), color, 2)
            cv2.putText(frame, label, (draw_x1, draw_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def calculate_center(detections, frame_width, frame_height, use_ball=True):
    """Oblicza centrum akcji, priorytetyzując piłkę lub zawodników/sędziów."""
    if use_ball:
        balls = [det for det in detections if int(det[5]) == 2]  # 'ball'
        if balls:
            x1, y1, x2, y2 = balls[0][:4]
            return (x1 + x2) / 2, (y1 + y2) / 2
    players_referees = [det for det in detections if int(det[5]) in [0, 1]]  # 'player' lub 'referee'
    if players_referees:
        centroids_x = [(det[0] + det[2]) / 2 for det in players_referees]
        centroids_y = [(det[1] + det[3]) / 2 for det in players_referees]
        return sum(centroids_x) / len(centroids_x), sum(centroids_y) / len(centroids_y)
    return frame_width // 2, frame_height // 2

def is_moving_ball(ball_det, prev_centroid, threshold):
    """Sprawdza, czy piłka się porusza na podstawie przesunięcia centroidu."""
    if prev_centroid is None:
        return True  # Pierwsza detekcja, uznajemy za ruch
    x1, y1, x2, y2 = ball_det[:4]
    current_centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
    distance = np.linalg.norm(np.array(current_centroid) - np.array(prev_centroid))
    return distance > threshold

prev_center_x, prev_center_y = width // 2, height // 2

# Pasek postępu
pbar = tqdm(total=total_frames, desc="Processing frames")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Zmniejszenie obrazu o połowę dla detekcji
    small_frame = resize_frame(frame, scale=0.5)
    small_height, small_width = small_frame.shape[:2]
    
    # Detekcja na pomniejszonym obrazie
    results = model(small_frame, classes=[0, 1, 2], device=device)  # Ogranicz do 'player', 'referee', 'ball'
    detections = results[0].boxes.data.cpu().numpy()
    
    # Przeskalowanie detekcji do oryginalnych wymiarów
    detections = scale_detections(detections, scale=0.5)
    
    # Filtrowanie detekcji piłek
    balls = [det for det in detections if int(det[5]) == 2]
    
    # Sprawdzenie, czy jest piłka w ruchu
    moving_balls = [ball for ball in balls if is_moving_ball(ball, prev_ball_centroid, movement_threshold)]
    
    if moving_balls:
        # Użyj pierwszej piłki w ruchu
        ball_det = moving_balls[0]
        frames_without_ball = 0
        last_ball_position = ball_det[:4]
        x1, y1, x2, y2 = last_ball_position
        prev_ball_centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
    elif balls:
        # Użyj statycznej piłki, jeśli nie ma piłki w ruchu
        ball_det = balls[0]
        last_ball_position = ball_det[:4]
        x1, y1, x2, y2 = last_ball_position
        prev_ball_centroid = ((x1 + x2) / 2, (y1 + y2) / 2)
        frames_without_ball += 1
    else:
        frames_without_ball += 1
        prev_ball_centroid = None  # Resetuj, jeśli nie ma detekcji
    
    # Decyzja, czy skupić się na piłce czy zawodnikach/sędziach
    if frames_without_ball <= frames_without_ball_threshold and last_ball_position is not None:
        # Użyj ostatniej znanej pozycji piłki
        x1, y1, x2, y2 = last_ball_position
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
    else:
        # Użyj pozycji zawodników/sędziów
        center_x, center_y = calculate_center(detections, width, height, use_ball=False)
    
    # Wygładzanie liniowe
    center_x = prev_center_x * (1 - smoothing_factor) + center_x * smoothing_factor
    center_y = prev_center_y * (1 - smoothing_factor) + center_y * smoothing_factor
    prev_center_x, prev_center_y = center_x, center_y
    
    # Wygładzanie średnią kroczącą
    positions_x.append(center_x)
    positions_y.append(center_y)
    if len(positions_x) > window_size:
        positions_x.pop(0)
        positions_y.pop(0)
    smoothed_center_x = sum(positions_x) / len(positions_x)
    smoothed_center_y = sum(positions_y) / len(positions_y)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    top = int(max(0, min(smoothed_center_y - crop_height / 2, height - crop_height)))
    right = left + crop_width
    bottom = top + crop_height
    
    # Przycięcie oryginalnego obrazu
    cropped_frame = frame[top:bottom, left:right]
    
    # Narysowanie detekcji na przyciętym obrazie
    draw_detections(cropped_frame, detections, left, top, right, bottom)
    
    # Skalowanie do 1280x720, jeśli kadr nie jest dokładny
    if cropped_frame.shape != (crop_height, crop_width, 3):
        cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height), interpolation=cv2.INTER_AREA)
    
    # Zapis klatki do tymczasowego pliku
    out.write(cropped_frame)
    
    # Aktualizacja paska postępu
    pbar.update(1)

# Zamknięcie paska postępu
pbar.close()

# Zwolnienie zasobów OpenCV
cap.release()
out.release()
cv2.destroyAllWindows()

# Dodanie dźwięku za pomocą moviepy
print("Adding audio to the output video...")
input_clip = VideoFileClip(input_path)
temp_clip = VideoFileClip(temp_video_path)

# Ustawienie dźwięku z oryginalnego wideo
final_clip = temp_clip.set_audio(input_clip.audio)

# Zapis finalnego wideo z dźwiękiem
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Zamknięcie klipów
input_clip.close()
temp_clip.close()
final_clip.close()

# Usunięcie tymczasowego pliku
os.remove(temp_video_path)
print("Output video with audio saved successfully.")

Using device: cuda


Processing frames: 100%|██████████| 1332/1332 [00:29<00:00, 44.92it/s]


Adding audio to the output video...
Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
Output video with audio saved successfully.


Kadrowanie od lewej do prawej - tylko zawodnicy


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from moviepy.editor import VideoFileClip
import os
from datetime import datetime
from tqdm import tqdm
import logging

# Wyłączenie szczegółowych logów YOLO
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# Sprawdzenie dostępności GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Wczytanie modelu YOLO
model = YOLO('yolov9c.pt').to(device)

# Mapowanie indeksów klas na nazwy
class_names = {0: "player", 1: "referee", 2: "ball"}

# Ścieżki do plików
input_path = '/media/512GB_ext/BurzaVSZryw_mini.mp4'
output_path = 'test.mp4'
temp_video_path = f'temp_output_{datetime.now().strftime("%Y%m%d_%H%M%S")}.mp4'

# Otwarcie wideo
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Docelowy rozmiar kadru (16:9, wysokość 720)
crop_height = height
crop_width = int(crop_height * 16 / 9)  # 1280

# Parametry wygładzania
window_size = 15
positions_x = []
smoothing_factor = 0.1

# Definicja stref: lewa 40%, środkowa 20%, prawa 40%
left_zone = 0.4 * width
middle_zone_start = 0.4 * width
middle_zone_end = 0.6 * width
right_zone = 0.6 * width

# Inicjalizacja zapisu tymczasowego wideo bez dźwięku
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(temp_video_path, fourcc, fps, (crop_width, crop_height))

def resize_frame(frame, scale=0.5):
    """Zmniejsza obraz o podany współczynnik skali."""
    return cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

def scale_detections(detections, scale):
    """Przeskalowuje współrzędne detekcji do oryginalnych wymiarów."""
    for det in detections:
        det[0] /= scale  # x1
        det[2] /= scale  # x2
        det[1] /= scale  # y1
        det[3] /= scale  # y2
    return detections

def draw_detections(frame, detections, left, top, right, bottom):
    """Rysuje bounding boxy i nazwy klas na przyciętym obrazie dla wszystkich klas."""
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if x1 < right and x2 > left and y1 < bottom and y2 > top:
            draw_x1 = int(max(x1 - left, 0))
            draw_y1 = int(max(y1 - top, 0))
            draw_x2 = int(min(x2 - left, crop_width))
            draw_y2 = int(min(y2 - top, crop_height))
            label = class_names[int(cls)]
            color = (0, 255, 0) if int(cls) == 2 else (255, 0, 0)
            cv2.rectangle(frame, (draw_x1, draw_y1), (draw_x2, draw_y2), color, 2)
            cv2.putText(frame, label, (draw_x1, draw_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def calculate_player_median(detections):
    """Oblicza medianę położenia zawodników w poziomie."""
    players = [det for det in detections if int(det[5]) == 0]  # 'player'
    if players:
        centroids_x = [(det[0] + det[2]) / 2 for det in players]
        return np.median(centroids_x)
    return None

prev_center_x = width // 2

# Pasek postępu
pbar = tqdm(total=total_frames, desc="Processing frames")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Zmniejszenie obrazu o połowę dla detekcji
    small_frame = resize_frame(frame, scale=0.5)
    small_height, small_width = small_frame.shape[:2]
    
    # Detekcja na pomniejszonym obrazie
    results = model(small_frame, classes=[0, 1, 2], device=device)
    detections = results[0].boxes.data.cpu().numpy()
    
    # Przeskalowanie detekcji do oryginalnych wymiarów
    detections = scale_detections(detections, scale=0.5)
    
    # Obliczenie mediany położenia zawodników
    median_x = calculate_player_median(detections)
    
    if median_x is not None:
        # Określenie docelowego centrum na podstawie stref
        if median_x < left_zone:
            target_center_x = crop_width / 2  # Lewa granica
        elif median_x > right_zone:
            target_center_x = width - crop_width / 2  # Prawa granica
        else:
            target_center_x = width / 2  # Środek
    else:
        target_center_x = width / 2
    
    # Wygładzanie liniowe
    center_x = prev_center_x * (1 - smoothing_factor) + target_center_x * smoothing_factor
    prev_center_x = center_x
    
    # Wygładzanie średnią kroczącą
    positions_x.append(center_x)
    if len(positions_x) > window_size:
        positions_x.pop(0)
    smoothed_center_x = sum(positions_x) / len(positions_x)
    
    # Obliczenie pozycji kadru
    left = int(max(0, min(smoothed_center_x - crop_width / 2, width - crop_width)))
    right = left + crop_width
    top = 0  # Kadr stały w pionie
    bottom = height
    
    # Przycięcie oryginalnego obrazu
    cropped_frame = frame[top:bottom, left:right]
    
    # Narysowanie detekcji na przyciętym obrazie
    draw_detections(cropped_frame, detections, left, top, right, bottom)
    
    # Skalowanie do 1280x720, jeśli kadr nie jest dokładny
    if cropped_frame.shape != (crop_height, crop_width, 3):
        cropped_frame = cv2.resize(cropped_frame, (crop_width, crop_height), interpolation=cv2.INTER_AREA)
    
    # Zapis klatki do tymczasowego pliku
    out.write(cropped_frame)
    
    # Aktualizacja paska postępu
    pbar.update(1)

# Zamknięcie paska postępu
pbar.close()

# Zwolnienie zasobów OpenCV
cap.release()
out.release()
cv2.destroyAllWindows()

# Dodanie dźwięku za pomocą moviepy
print("Adding audio to the output video...")
input_clip = VideoFileClip(input_path)
temp_clip = VideoFileClip(temp_video_path)

# Ustawienie dźwięku z oryginalnego wideo
final_clip = temp_clip.set_audio(input_clip.audio)

# Zapis finalnego wideo z dźwiękiem
final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')

# Zamknięcie klipów
input_clip.close()
temp_clip.close()
final_clip.close()

# Usunięcie tymczasowego pliku
os.remove(temp_video_path)
print("Output video with audio saved successfully.")

Using device: cuda


Processing frames: 100%|██████████| 959/959 [00:28<00:00, 33.54it/s]


Adding audio to the output video...
Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
Output video with audio saved successfully.
